In [6]:
import json
import pymorphy2
import pandas as pd
import numpy as np

# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 1 

## Работу выполняли:

## Корнеев А.В.
## Щербакова П.В.

### ИАД-2

### 1. Сбор данных

В задании 1 требуется извлечь прогноз погоды на 10 дней для произвольно выбранного города России с сайта gismeteo.ru.
Резльтатом сбора данных должна быть таблица со следующими строками:
* минимальная температура
* максимальна температура
* скорость ветра
* уровень осадков 

В столбцах таблицы должны быть даты и дни недели.

Был выбран город Москва https://www.gismeteo.ru/weather-moscow-4368/10-days/. 
Для того чтобы извлечь прогноз погоды с сайта gismeteo были использованы инструменты разработчика браузера Google Chrome и кроулер веб-страниц scrapy.
С помощью инструментов разработчика было определено, в каких элементах html кода страницы находятся необходимые данные. Далее, с помощью scrapy, встроенного в него css-селектора данные о прогнозе были извлечены в json-файл. 

(Scrapy используется отдельно от jupyter notebook и вызывается из терминала)

#### Scrapy-скрипт:
```python
import scrapy
import re

class WeatherSpider(scrapy.Spider):
    name = 'weather'
    allowed_domains = ['gismeteo.ru']
    start_urls = ['https://www.gismeteo.ru/weather-moscow-4368/10-days/']

    def parse(self, response):
        days = response.css('.forecast_frame .w_date__day::text').extract()
        dates = response.css('.forecast_frame .w_date__date::text').re('\d\d')
        dd = list(map(lambda x: x[0]+' '+x[1]+'.02',list(zip(days,dates))))
        maxt = response.css('.chart__temperature .maxt::text').extract()
        maxt_pure = list(map(lambda x: int(x.replace('−','-')),maxt))
        mint = response.css('.chart__temperature .mint::text').extract()
        mint_pure = list(map(lambda x: int(x.replace('−','-')),mint))
        hum = response.css('.w_precipitation__value::text').extract()
        hum_pure = list(map(lambda x: float(x.split()[0].replace(',','.')),hum))
        wind = response.css('.widget__row_table:nth-child(5) .w_wind__warning::text').extract()
        wind_pure = list(map(lambda x: int(x.split()[0]),wind))
        yield {'dd':dd,'Максимальная температура':maxt_pure,'Минимальная температура':mint_pure,'Уровень осадков':hum_pure,'Скорость ветра':wind_pure}
```

#### Пояснение: 
На примере данных о максимальной температуре поясним смысл скрипта. 
Данные о максимальной температуре находятся в html коде страницы в элементах класса maxt, которые в свою очередь находятся в элементах класса chart_ _temperature, чтобы извлечь эти данные применяем css-селектор:
```python
maxt = response.css('.chart__temperature .maxt::text').extract()
```
Данные были извлечены в виде строк и содержат лишние символы, в случае с максимальной температурой, для отрицательных показателей перед цифрами стоит "длинный минус", в таком виде приведение к типу int не сработает, поэтому заменяем этот символ на обычный минус и приводит даннык к типу int.
```python
maxt_pure = list(map(lambda x: int(x.replace('−','-')),maxt))
```
Для извлечения остальных данных проводятся аналогичные операции.

In [7]:
data = json.load(open('weather_data.json'))

In [8]:
data = data[0]
headers = data['dd']
del data['dd']

In [9]:
data['Средняя температура']=list(map(lambda x: int(x),np.round((np.array(data['Минимальная температура'])+np.array(data['Максимальная температура']))/2)))

In [10]:
data

{'Максимальная температура': [-7, -5, -3, -4, -5, -7, -10, -11, -4, -5],
 'Минимальная температура': [-11, -8, -5, -6, -7, -11, -16, -18, -11, -11],
 'Скорость ветра': [6, 8, 8, 8, 7, 7, 6, 13, 15, 12],
 'Средняя температура': [-9, -6, -4, -5, -6, -9, -13, -14, -8, -8],
 'Уровень осадков': [0.5, 0.8, 2.5, 2.0, 4.0, 0.2, 0.0, 2.0, 7.4, 0.6]}

In [11]:
headers

['Чт 15.02',
 'Пт 16.02',
 'Сб 17.02',
 'Вс 18.02',
 'Пн 19.02',
 'Вт 20.02',
 'Ср 21.02',
 'Чт 22.02',
 'Пт 23.02',
 'Сб 24.02']

In [12]:
df = pd.DataFrame.from_dict(data).transpose()
df.columns = headers

In [13]:
df = df.reindex(['Максимальная температура','Минимальная температура','Средняя температура','Скорость ветра','Уровень осадков'])

Данные о прогнозе были извлечены из json-файла и помещены в pandas DataFrame. Также в таблицу была добавлена информация о средней дневной температуре, она понадобится в следующем задании для генерации прогноза по шаблону.

In [14]:
df

,Чт 15.02,Пт 16.02,Сб 17.02,Вс 18.02,Пн 19.02,Вт 20.02,Ср 21.02,Чт 22.02,Пт 23.02,Сб 24.02
Максимальная температура,-7.0,-5.0,-3.0,-4.0,-5.0,-7.0,-10.0,-11.0,-4.0,-5.0
Минимальная температура,-11.0,-8.0,-5.0,-6.0,-7.0,-11.0,-16.0,-18.0,-11.0,-11.0
Средняя температура,-9.0,-6.0,-4.0,-5.0,-6.0,-9.0,-13.0,-14.0,-8.0,-8.0
Скорость ветра,6.0,8.0,8.0,8.0,7.0,7.0,6.0,13.0,15.0,12.0
Уровень осадков,0.5,0.8,2.5,2.0,4.0,0.2,0.0,2.0,7.4,0.6


### 2. Генератор описания прогноза погоды

Прогноз погоды должен состоять из следующих (или подобным им) предложений, генерируемых по шаблонам (ниже три шаблона):
* В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    * *В четверг в НазваниеГорода потеплеет на 7 градусов по сравнению со средой*
* Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    * *Скорость ветра изменится на 3 единицы в понедельник по сравнению с пятницей*
* Уровень осадков повысится / понизится на X единиц за Y дней. 
    * *Уровень осадков понится на 3.85 единиц за 7 дней*

Вместо НазваниеГорода подставьте название выбранного вами города, используя фунцкии для согласования существительных с предлогами. 

Для генерации предложений по вышеперичисленным шаблонам были написаны функции, которые используют методы пакета pymorphy2 для согласования слов в предложении.

In [15]:
days_dict = {'Пн':'понедельник','Вт':'вторник','Ср':'среда','Чт':'четверг','Пт':'пятница','Сб':'суббота','Вс':'воскресенье'}

In [16]:
def morph_analyze(word):
    morph = pymorphy2.MorphAnalyzer()
    return morph.parse(word)[0]

Для генерации предложения по шаблону требуется согласовывать слова с предлогами и числительными.
В частности, для генерации по шаблону "В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2" требуется согласовывать предлоги "в","с" с названиями дней недели. Для этого была создана функция __prep_day_agree(Prep,Day)__, котораня принимает на вход предлог и слово с ним согласовываемое и возврящает строку: предлог + согласованное слово. 
В случае с предлогом "в", если он согласуется со словом "вторник", то предлог ставится в дополнительную форму "во", в остальных случаях согласовываемое слово ставится в винительный падеж.

В случае с предлогом "с", если он согласуется со словом "вторник" или "среда", то предлог ставится в дополнительную форму "со", а слово в творительный падеж, в остальных случаях предлог остается неизменным, а согласовываемое слово ставится в творительный падеж.

Для постановки слова в необходимый падеж используется метод пакета pymorphy2: inflect().

Выбор падежей обоснован контекстом, в котором используются слова.

In [17]:
def prep_day_agree(Prep,Day):
    if Prep.lower() == 'в':
        if Day == 'вторник':
            return morph_analyze(Prep).inflect({'Vpre'}).word +' '+  morph_analyze(Day).inflect({'accs'}).word
        else:
             return Prep +' '+  morph_analyze(Day).inflect({'accs'}).word   
    if Prep.lower() == 'с':
        if Day == 'вторник' or Day == 'среда':
            return morph_analyze(Prep).inflect({'Vpre'}).word +' '+  morph_analyze(Day).inflect({'ablt'}).word
        else:
             return Prep +' '+  morph_analyze(Day).inflect({'ablt'}).word   

Функция __num_agree_word(num,word)__ согласовывает слово с числительным. Принимает на вход числительное и слово, возвращает согласованное слово. Надо заметить, что числительное "1", обрабатывается отдельно, в случае числительного "1", согласовываемое слово ставится в винительный падеж (хотим получить "на 1 единицу", а не "на 1 единица"), в остальный случаях используется метод пакета pymorphy2 make_agree_with_number().

In [21]:
def num_agree_word(num,word):
    if abs(num) == 1:
        return morph_analyze(word).inflect({'accs'}).word
    else:
        return morph_analyze(word).make_agree_with_number(abs(num)).word

In [36]:
num_agree_word(1,'единица')

'единицу'

Функция __gen_template_temp(D1,D2,City_Name)__ генерирует предложение по шаблону "В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2". На вход получает "заголовки" из таблицы с прогнозом (например D1 = "Чт 15.02" D2 = "Пт 16.02"), а также название города. Возвращает предложение сгенерированное по шаблону.

Стоит отметить что в шаблон также былы добавлены календарные даты, чтобы можно было отличать разные дни с одинаковым днем недели. Также, в случае если для сравниваемых дней прогноз температуры одинаков, возвращается предложение вида: "В день1 в НазваниеГорода температура не изменится по сравнению с день2 (X градусов )."

#### Важно:
Для генерации предложения используется средняя дневная температура из таблицы, полученная путем усреднения и округления максимальной и минимальной температуры для каждого дня.

Внутри функции проводится сравнение средней температуры в день1 и день2, и в зависимости от результата сравнения выбирается слово потоеплеет/похолодает.

Для генерации итогового предложения используются вышеописанные функции для согласования слов с предлогами и числительными.

In [34]:
def gen_template_temp(D1,D2,City_Name):
    D1_day = days_dict[D1.split()[0]]
    D1_date = D1.split()[1]
    D2_day = days_dict[D2.split()[0]]
    D2_date = D2.split()[1]
    
    temp_trend = ''
    City_Name_agreed = morph_analyze(City_Name).inflect({'loct'}).word.title()
    
    D1_med_temp = int(df[D1]['Средняя температура'])
    D2_med_temp = int(df[D2]['Средняя температура'])
    
    if D1_med_temp > D2_med_temp:
        temp_trend = "потеплеет"
    elif D1_med_temp < D2_med_temp:
        temp_trend = "похолодает"
    else:
        #import pdb; pdb.set_trace()
        return prep_day_agree('В',D1_day)+' '+D1_date+' в '+City_Name_agreed+' температура не изменится по сравнению '+ prep_day_agree('с',D2_day)+' '+D2_date+' ('+str(D1_med_temp)+' '+num_agree_word(D1_med_temp,'градус')+' ).'
    
    return prep_day_agree('В',D1_day).title()+' '+D1_date+' в '+City_Name_agreed+' '+temp_trend+' '+'на '+str(abs(D1_med_temp-D2_med_temp))+' '+num_agree_word(abs(D1_med_temp-D2_med_temp),'градус')+' по сравнению'+' '+prep_day_agree('с',D2_day)+' '+D2_date+'.'

Пример работы функции __gen_template_temp(D1,D2,City_Name)__, сравним температуру в субботу 24.02 со всеми днями из таблицы.

In [35]:
for day in df.columns:
    print(gen_template_temp(df.columns[-1],day,'Москва'))

В Субботу 24.02 в Москве потеплеет на 1 градус по сравнению с четвергом 15.02.
В Субботу 24.02 в Москве похолодает на 2 градуса по сравнению с пятницей 16.02.
В Субботу 24.02 в Москве похолодает на 4 градуса по сравнению с субботой 17.02.
В Субботу 24.02 в Москве похолодает на 3 градуса по сравнению с воскресеньем 18.02.
В Субботу 24.02 в Москве похолодает на 2 градуса по сравнению с понедельником 19.02.
В Субботу 24.02 в Москве потеплеет на 1 градус по сравнению со вторником 20.02.
В Субботу 24.02 в Москве потеплеет на 5 градусов по сравнению со средой 21.02.
В Субботу 24.02 в Москве потеплеет на 6 градусов по сравнению с четвергом 22.02.
В субботу 24.02 в Москве температура не изменится по сравнению с пятницей 23.02 (-8 градусов ).
В субботу 24.02 в Москве температура не изменится по сравнению с субботой 24.02 (-8 градусов ).


In [20]:
df

,Чт 15.02,Пт 16.02,Сб 17.02,Вс 18.02,Пн 19.02,Вт 20.02,Ср 21.02,Чт 22.02,Пт 23.02,Сб 24.02
Максимальная температура,-7.0,-5.0,-3.0,-4.0,-5.0,-7.0,-10.0,-11.0,-4.0,-5.0
Минимальная температура,-11.0,-8.0,-5.0,-6.0,-7.0,-11.0,-16.0,-18.0,-11.0,-11.0
Средняя температура,-9.0,-6.0,-4.0,-5.0,-6.0,-9.0,-13.0,-14.0,-8.0,-8.0
Скорость ветра,6.0,8.0,8.0,8.0,7.0,7.0,6.0,13.0,15.0,12.0
Уровень осадков,0.5,0.8,2.5,2.0,4.0,0.2,0.0,2.0,7.4,0.6


Функция __gen_template_wind(D1,D2)__ работает аналогично предыдущей и генерирует предложение по шаблону "Скорость ветра изменится на X единиц в день1 по сравнению с день2." (вместо слова "единиц" использовали "метров в секунду"). На вход получает "заголовки" из таблицы с прогнозом (например D1 = "Чт 15.02" D2 = "Пт 16.02"). Возвращает предложение сгенерированное по шаблону.

С помощью принятых на вход аргументов, функция выбирает из таблицы соответсвующие записи о скорости ветра из таблицы с прогнозом и, пользуясь  функциями для согласования слов с предлогами и числительными, генерирует итоговое предложение.

In [22]:
def gen_template_wind(D1,D2):
    D1_day = days_dict[D1.split()[0]]
    D1_date = D1.split()[1]
    D2_day = days_dict[D2.split()[0]]
    D2_date = D2.split()[1]
    
    D1_wind = int(df[D1]['Скорость ветра'])
    D2_wind = int(df[D2]['Скорость ветра'])
    
    return 'Скорость ветра изменится на '+str(D1_wind-D2_wind)+' '+ num_agree_word(D1_wind-D2_wind,'метр')+' в секунду '+' '+prep_day_agree('в',D1_day)+' '+D1_date+' по сравнению'+' '+prep_day_agree('с',D2_day)+' '+D2_date+'.'

Пример работы функции __gen_template_wind(D1,D2)__, сравним скорость ветра в субботу 24.02 со всеми днями из таблицы.

In [23]:
for day in df.columns:
    print(gen_template_wind(df.columns[-1],day))

Скорость ветра изменится на 6 метров в секунду  в субботу 24.02 по сравнению с четвергом 15.02.
Скорость ветра изменится на 4 метра в секунду  в субботу 24.02 по сравнению с пятницей 16.02.
Скорость ветра изменится на 4 метра в секунду  в субботу 24.02 по сравнению с субботой 17.02.
Скорость ветра изменится на 4 метра в секунду  в субботу 24.02 по сравнению с воскресеньем 18.02.
Скорость ветра изменится на 5 метров в секунду  в субботу 24.02 по сравнению с понедельником 19.02.
Скорость ветра изменится на 5 метров в секунду  в субботу 24.02 по сравнению со вторником 20.02.
Скорость ветра изменится на 6 метров в секунду  в субботу 24.02 по сравнению со средой 21.02.
Скорость ветра изменится на -1 метр в секунду  в субботу 24.02 по сравнению с четвергом 22.02.
Скорость ветра изменится на -3 метра в секунду  в субботу 24.02 по сравнению с пятницей 23.02.
Скорость ветра изменится на 0 метров в секунду  в субботу 24.02 по сравнению с субботой 24.02.


In [24]:
df

,Чт 15.02,Пт 16.02,Сб 17.02,Вс 18.02,Пн 19.02,Вт 20.02,Ср 21.02,Чт 22.02,Пт 23.02,Сб 24.02
Максимальная температура,-7.0,-5.0,-3.0,-4.0,-5.0,-7.0,-10.0,-11.0,-4.0,-5.0
Минимальная температура,-11.0,-8.0,-5.0,-6.0,-7.0,-11.0,-16.0,-18.0,-11.0,-11.0
Средняя температура,-9.0,-6.0,-4.0,-5.0,-6.0,-9.0,-13.0,-14.0,-8.0,-8.0
Скорость ветра,6.0,8.0,8.0,8.0,7.0,7.0,6.0,13.0,15.0,12.0
Уровень осадков,0.5,0.8,2.5,2.0,4.0,0.2,0.0,2.0,7.4,0.6


In [25]:
df.columns.get_loc('Пт 16.02')

1

In [26]:
df.columns.shape[0]

10

Уровень осадков повысится / понизится на X единиц за Y дней.

Для генерации предложений по шаблону "Уровень осадков повысится / понизится на X единиц за Y дней." потребовалась отдельная функция для согласования существительный с числительными. 
Дело в том, что для описания данных об уровне осадков могут использоваться как целые, так и дробные числа, а по правилам русского языка при согласовании смешанного числительно с существительным, существительным управляет дробь, то есть говорим 1,56 единицы (одна целая пятьдесят шесть сотых единицы), а не 1,56 единиц.

Поэтому фунция __num_agree_word_prec(num,word)__ согласовывает слово с чилительным, проверяя, является ли числительное целым или нет, и в случае если оно является целым, согласование происходит при помощи make_agree_with_number(), в противном случае согласовываемое слово ставится в родительный падеж.

In [27]:
def num_agree_word_prec(num,word):
    if num.is_integer():
        return morph_analyze(word).make_agree_with_number(int(num)).word
    else:
        return morph_analyze(word).inflect({'gent'}).word

Функция __gen_template_prec(D,Period)__ генерирует предложение по шаблону "Уровень осадков повысится / понизится на X единиц за Y дней." (вместо "единиц" использовали "миллиметры", также в конец предложения добавили относительного какого дня проводятся измерения). На вход принимает два аргумента: D - "загловок" из таблицы, Period - Y из шаблона. Возвращает предложение сгенерированное по шаблону, однако, если уровень осадков не меняется через укзанный период Y, то возвращается предложение вида: Уровень осадков не изменится за Y дней относительно день1.

Внутри функции, с помощью полученных на вход аргументов из таблицы выбираются данные об уровне осадков за два дня, в день D, и в день через Period дней от D. Вычисляется разница между уровнями осадков в эти дни, разница уровней осадков сравнивается с нулем, если она больше нуля, то будет выбрано слово "повысится", в противном случае "понизится".

In [28]:
def gen_template_prec(D,Period):
    offset = df.columns.get_loc(D)+Period
    if (offset) >= df.columns.shape[0]:
        return 'Недостаточно данных в таблице'
    
    prec_diff = df[df.columns[offset]]['Уровень осадков']-df[D]['Уровень осадков']
    prec_trend = ''
    
    if prec_diff > 0:
        prec_trend = 'повысится'
    elif prec_diff < 0:
        prec_trend = 'понизится'
    else:
        return 'Уровень осадков не изменится за '+str(Period)+' '+morph_analyze('день').make_agree_with_number(Period).word+' относительно '+D+'.'
    return 'Уровень осадков '+prec_trend+' на '+str(abs(prec_diff))+' '+num_agree_word_prec(abs(prec_diff),'миллиметр')+' за '+str(Period)+' '+morph_analyze('день').make_agree_with_number(Period).word+' относительно '+D+'.'

Пример работы функции __gen_template_prec(D,Period)__

In [37]:
for i in range(0,10):
    print(gen_template_prec(df.columns[0],i))

Уровень осадков не изменится за 0 дней относительно Чт 15.02.
Уровень осадков повысится на 0.3 миллиметра за 1 день относительно Чт 15.02.
Уровень осадков повысится на 2.0 миллиметра за 2 дня относительно Чт 15.02.
Уровень осадков повысится на 1.5 миллиметра за 3 дня относительно Чт 15.02.
Уровень осадков повысится на 3.5 миллиметра за 4 дня относительно Чт 15.02.
Уровень осадков понизится на 0.3 миллиметра за 5 дней относительно Чт 15.02.
Уровень осадков понизится на 0.5 миллиметра за 6 дней относительно Чт 15.02.
Уровень осадков повысится на 1.5 миллиметра за 7 дней относительно Чт 15.02.
Уровень осадков повысится на 6.9 миллиметра за 8 дней относительно Чт 15.02.
Уровень осадков повысится на 0.1 миллиметра за 9 дней относительно Чт 15.02.


In [38]:
df

,Чт 15.02,Пт 16.02,Сб 17.02,Вс 18.02,Пн 19.02,Вт 20.02,Ср 21.02,Чт 22.02,Пт 23.02,Сб 24.02
Максимальная температура,-7.0,-5.0,-3.0,-4.0,-5.0,-7.0,-10.0,-11.0,-4.0,-5.0
Минимальная температура,-11.0,-8.0,-5.0,-6.0,-7.0,-11.0,-16.0,-18.0,-11.0,-11.0
Средняя температура,-9.0,-6.0,-4.0,-5.0,-6.0,-9.0,-13.0,-14.0,-8.0,-8.0
Скорость ветра,6.0,8.0,8.0,8.0,7.0,7.0,6.0,13.0,15.0,12.0
Уровень осадков,0.5,0.8,2.5,2.0,4.0,0.2,0.0,2.0,7.4,0.6


### 3. Ответьте на вопросы

#### В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? 

Ответ: Генерирование текста по шаблонам используется почти во всех задачах, целью которых является сообщение самой важной информации о предмете/ситуации. Например, составление расписания электричек, описание загруженности дорог, описание товаров по характеристикам. Также это может применяться для составления различных отчетов, документов и даже электронных писем-рассылок, структура которых заранее известна и не отличается большой сложностью.

#### В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?

Ответ: Генерировать текст об изменении числовых показателей по шаблонам может пригодиться в повседневных задачах составления прогноза погоды, курса валют, описания наличия мест на парковках в разных частях города и тд.

#### Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче?

Ответ: Знание синтаксической структуры предложения - основной способ разнообразить предложения-шаблоны. Например, вместо простых предложений можно сделать сложносочиненное/сложноподчиненное предложение : "В четверг в Москве похолодает, а скорость ветра будет 15 км/ч". Чтобы объединить несколько шаблонных предложений, можно использовать однородные члены предложения : "В Москве и Петербурге потеплеет на 8 градусов". Также, зная о довольно свободном расположении слов в предложении ( у членов предложения в русском языке нет "конкретной позиции"), можно изменить получившиеся предложения, чтобы они звучали естественнее. Вместо "Скорость ветра изменится на 6 метров в секунду  в субботу 24.02 по сравнению с четвергом 15.02." можно получить "По сравнению с четвергом 16.02 скорость ветра в субботу 24.02 изменится на 6 метров в секунду".
